In [1]:
#Install the required libraries
!pip install accelerate
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
import os
import torch
import accelerate
from datasets import load_dataset,Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel,prepare_model_for_kbit_training,get_peft_model
from trl import SFTTrainer
import json
import numpy as np
from huggingface_hub import notebook_login

In [3]:
# Hugging face access token will be required to access the model
from huggingface_hub import login
login(token="ADD YOUR TOKEN")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
model_id = "Huma97/llama2-EquityAdvisor"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [5]:
#modify the data type used during computation by setting the bnb_4bit_compute_dtype to a different value, such as torch.bfloat16.

bnb_config = BitsAndBytesConfig(
    # load models in 4-bit quantization.
    load_in_4bit=True,
    #The integration also recommends using the nested quantization technique for even greater memory efficiency without sacrificing performance.
    bnb_4bit_use_double_quant=True,
    #The NF4 data type is designed for weights initialized using a normal distribution
    bnb_4bit_quant_type="nf4",
    #modify the data type used during computation by setting the bnb_4bit_compute_dtype to a different value, such as torch.bfloat16.
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id ,

                                          torch_dtype="auto")
tokenizer.use_default_system_prompt = False

# This line of code sets the tokenizer's pad token to be the same as its EOS token.
#This means that the same token will be used both to pad shorter sequences and to signify the end of a sequence.
# Why would you do this? In some models and contexts, it can be beneficial to treat the end of a sequence and padding as the same,
#especially in models where the distinction between padding and the end of a sequence is not crucial or might
#interfere with how the model processes the text. For example, in some generative models,
#having distinct padding and EOS tokens might lead to unwanted behavior during text generation,
#and using the same token for both can mitigate such issues.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#Load the model
model = AutoModelForCausalLM.from_pretrained(
                                         model_id,
                                         quantization_config=bnb_config,
                                         device_map='auto')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

Loading adapter weights from Huma97/llama2-EquityAdvisor led to unexpected keys not found in the model:  ['base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight', 'base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.laye

In [8]:

prompt = f"""
your task is to generate the trading signal recommendation for a trader involving less complex financial terms in the response. Your response should be more focused on the recommendation. Do not use much technical terminologies in the response and its rational.
Given the EMA55_Signal: -1, EMA9_Signal: -3, MACD_Signal:3, RSI_Signal: 2 with news feature: oil prices fell and events impact =- 5.33 at is the trading recommendation for the following day in the oil sector to an active trader?

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt').to(device)
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=500,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
# print(f'BASELINE HUMAN SUMMARY:\n{answer}\n')
# print(dash_line)
print(f'MODEL GENERATION -:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

your task is to generate the trading signal recommendation for a trader involving less complex financial terms in the response. Your response should be more focused on the recommendation. Do not use much technical terminologies in the response and its rational.
Given the EMA55_Signal: -1, EMA9_Signal: -3, MACD_Signal:3, RSI_Signal: 2 with news feature: oil prices fell and events impact =- 5.33 at is the trading recommendation for the following day in the oil sector to an active trader?

Answer:

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

your task is to generate the trading signal recommendation for a trader involving less complex financial terms in the response. Your response should be more focused on the recommendation. Do not use much technical terminologies in the response and its r